## BPC/SU/II Survey

### Install Packages, Load Data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from textwrap import wrap

In [2]:
codebook = pd.read_csv('https://raw.githubusercontent.com/rachelorey/Dec23-survey/main/2312082_BPC_levels_codebook.csv', encoding='utf-8')
data = pd.read_csv('https://raw.githubusercontent.com/rachelorey/Dec23-survey/main/2312082_BPC_raw_data.csv', encoding='utf-8')
q_codebook = pd.read_csv('https://raw.githubusercontent.com/rachelorey/Dec23-survey/main/2312082_BPC_question_codebook.csv', encoding='utf-8', index_col = 'qid')

# q_codebook into dict
q_codebook = q_codebook.to_dict().get('qidFull')

# display(data)

In [3]:
nulldata = pd.DataFrame(data.isnull().sum(),columns=["Sum_Null"]) ##2203 rows in entire dataset
nulldata.sort_values(by=["Sum_Null"],ascending=False,inplace=True)

# nulldata[nulldata["Sum_Null"]>0].head(20)

#BPC5: 'Do you believe social media platforms are doing too little or too much to protect users from harmful content?'
#BPC4: You indicated that you are not interested in learning about how elections are run in the United States , why is that?

## Get Percents by Demo for Scaled Questions
#### BPC21, BPC3

In [4]:
def weighted_counts(x):
    # x is a DataFrame of grouped values including the 'wts' column for weights
    total_weight = x['wts'].sum()
    return total_weight

def getdemo_scaled(demo,q,collapse_scales=False):
    # Extracting demo labels
    labels = dict(zip(codebook[codebook['question'] == demo]['value'], codebook[codebook['question'] == demo]['code']))

    # Filtering out the relevant rows from the codebook for BPCX responses
    bpcx_codebook = codebook[codebook['question'].str.contains(q)]

    # Creating a dictionary to map response values to their meanings
    value_to_meaning = dict(zip(bpcx_codebook['value'], bpcx_codebook['code']))

    # List to hold the reshaped tables
    long_format_tables = []

    # Iterate through each BPCX question
    for question in data.filter(regex='^'+q).columns:

##ADDING WEIGHTS
        # Group by 'demo' and 'question', then apply the weighted counts calculation
        summary_table = data.groupby([demo,question]).apply(weighted_counts).unstack()
    
        # Rename index using the labels dictionary
        summary_table = summary_table.rename(index=labels)

        # Rename columns using the value_to_meaning dictionary
        summary_table = summary_table.rename(columns=value_to_meaning)

        # Reset index to make demo a column
        summary_table = summary_table.reset_index()

        # Melt the DataFrame to long format
        melted = summary_table.melt(id_vars=demo, var_name='Response', value_name='Count')
        melted['Question'] = question

        # Replace the question shorthand with specific concern
        melted['Q_Text'] = q_codebook.get(question, question)  # Fallback to question ID if not found

        # Append to the list
        long_format_tables.append(melted)

    # Concatenate all the long format tables into one
    combined_long_table = pd.concat(long_format_tables)
    try:
        combined_long_table["Q_Text"] = combined_long_table["Q_Text"].str.split(' --- ', expand=True)[1].fillna(combined_long_table["Q_Text"])
    except:
        pass
    

    response_mapping = {
        'Very concerned': 'Concerned',
        'Somewhat concerned': 'Concerned',
        'Not too concerned': 'Not Concerned',
        'Not at all concerned': 'Not Concerned',
        "Don't know/No opinion": "Don't know/No opinion",
        'Very important': 'Important',
        'Somewhat important': 'Important',
        'Not too important': 'Not important',
        'Not at all important': 'Not important',
        "Far too little":"Too little",
        "Far too much":"Too much"        
    }
    
    if collapse_scales: #if function says to collapse scales (ie take out very, somehwat)
        if ~combined_long_table["Response"].str.lower().str.contains("confident").any(): #if Response values don't include "confident"
            # Apply the mapping to collapse response categories
            combined_long_table['Response'] = combined_long_table['Response'].map(response_mapping)

    # Pivot the table to wide format
    wide_format_table = combined_long_table.pivot_table(index=['Q_Text','Response'], 
                                                        columns=[demo], 
                                                        values='Count',
                                                        fill_value=0)

    wide_format_table.reset_index(inplace=True)

    for q_level in wide_format_table["Q_Text"].unique():
        # Mask for the rows that correspond to the question
        question_mask = wide_format_table["Q_Text"] == q_level

        # Select the relevant data for the current question
        temp = wide_format_table.loc[question_mask, wide_format_table.columns[2:]]

        # Calculate the totals for each column (for the current question)
        totals = temp.sum()

        # Use .loc to update the original dataframe
        wide_format_table.loc[question_mask, temp.columns] = temp / totals.values

    
    # GET TOTAL PERCENTS
    
    # Get total responses by Q_Text
    total_counts = combined_long_table.groupby(['Q_Text'])['Count'].sum().reset_index(name='Total Count')

    overall_percent = (combined_long_table.groupby(["Response","Q_Text"])['Count'].sum()/combined_long_table.groupby(['Q_Text'])['Count'].sum()[0]).reset_index()
    overall_percent.rename({"Count":"Overall"},axis=1,inplace=True)
    
    
    wide_format_table = pd.merge(wide_format_table, overall_percent, on=["Response", 'Q_Text'])
    
    wide_format_table.to_csv("C://Users//rorey//OneDrive - Bipartisan Policy Center//Elections Project//Research//BPC SU II Joint Survey Dec 23//programming//files//"+q+"_"+demo+".csv",index=False)
    
#     display(wide_format_table)

    return(combined_long_table,wide_format_table)

In [49]:
for q in ["BPC12","BPC13","BPC14","BPC16"]:
    getdemo_scaled("age",q,False)

C:\Users\rorey\AppData\Local\Temp\ipykernel_15840\1706240069.py:99: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  overall_percent = (combined_long_table.groupby(["Response","Q_Text"])['Count'].sum()/combined_long_table.groupby(['Q_Text'])['Count'].sum()[0]).reset_index()
C:\Users\rorey\AppData\Local\Temp\ipykernel_15840\1706240069.py:99: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  overall_percent = (combined_long_table.groupby(["Response","Q_Text"])['Count'].sum()/combined_long_table.groupby(['Q_Text'])['Count'].sum()[0]).reset_index()
C:\Users\rorey\AppData\Local\Temp\ipykernel_15840\1706240069.py:99: FutureWarnin

In [47]:
getdemo_scaled("xpid3","BPC7",False) #BPC7 BPC5

C:\Users\rorey\AppData\Local\Temp\ipykernel_15840\1706240069.py:99: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  overall_percent = (combined_long_table.groupby(["Response","Q_Text"])['Count'].sum()/combined_long_table.groupby(['Q_Text'])['Count'].sum()[0]).reset_index()


(                 xpid3                                           Response  \
 0   PID: Dem (no lean)  Mainly the responsibility of social media plat...   
 1   PID: Ind (no lean)  Mainly the responsibility of social media plat...   
 2   PID: Rep (no lean)  Mainly the responsibility of social media plat...   
 3   PID: Dem (no lean)  Social media platforms and government should s...   
 4   PID: Ind (no lean)  Social media platforms and government should s...   
 5   PID: Rep (no lean)  Social media platforms and government should s...   
 6   PID: Dem (no lean)            Mainly the responsibility of government   
 7   PID: Ind (no lean)            Mainly the responsibility of government   
 8   PID: Rep (no lean)            Mainly the responsibility of government   
 9   PID: Dem (no lean)                            Neither are responsible   
 10  PID: Ind (no lean)                            Neither are responsible   
 11  PID: Rep (no lean)                            Neither are r

In [40]:
for q in ["BPC4","BPC7"]:
    getdemo_scaled("xpid3",q,False)

C:\Users\rorey\AppData\Local\Temp\ipykernel_15840\1706240069.py:99: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  overall_percent = (combined_long_table.groupby(["Response","Q_Text"])['Count'].sum()/combined_long_table.groupby(['Q_Text'])['Count'].sum()[0]).reset_index()
C:\Users\rorey\AppData\Local\Temp\ipykernel_15840\1706240069.py:99: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  overall_percent = (combined_long_table.groupby(["Response","Q_Text"])['Count'].sum()/combined_long_table.groupby(['Q_Text'])['Count'].sum()[0]).reset_index()


In [8]:
##RUNS FOR ALL CONFIDENCE QUESTIONS

res = pd.DataFrame(columns=['Q_Text', 'Response', 'PID: Dem (no lean)', 'PID: Ind (no lean)','PID: Rep (no lean)','Overall'])

dataframes_to_concat = [res]

for que in ["BPC17","BPC18","BPC19","BPC20"]:
    combined_long_table,wide_format_table = getdemo_scaled("xpid3",que,False)
    dataframes_to_concat.append(wide_format_table)

res = pd.concat(dataframes_to_concat, ignore_index=True)

res.to_csv("C://Users//rorey//OneDrive - Bipartisan Policy Center//Elections Project//Research//BPC SU II Joint Survey Dec 23//programming//files//BPC17-20.csv",index=False)

C:\Users\rorey\AppData\Local\Temp\ipykernel_15840\2643580208.py:103: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  overall_percent = (combined_long_table.groupby(["Response","Q_Text"])['Count'].sum()/combined_long_table.groupby(['Q_Text'])['Count'].sum()[0]).reset_index()
C:\Users\rorey\AppData\Local\Temp\ipykernel_15840\2643580208.py:103: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  overall_percent = (combined_long_table.groupby(["Response","Q_Text"])['Count'].sum()/combined_long_table.groupby(['Q_Text'])['Count'].sum()[0]).reset_index()
C:\Users\rorey\AppData\Local\Temp\ipykernel_15840\2643580208.py:103: FutureWar

## Getting Percents by Question and Demo for "Select All That Apply" Questions

In [62]:
##FUNCTION to get breakdown in responses for "select all that apply" question by demo

def get_demo_percents(demo,q):

    # Extracting political party labels
    demo_labels = dict(zip(codebook[codebook['question'] == demo]['value'], codebook[codebook['question'] == demo]['code']))

    # Filtering out the relevant rows from the codebook for BPCX responses
    bpcx_codebook = codebook[codebook['question'].str.contains(q)]

    # Creating a dictionary to map response values to their meanings
    value_to_meaning = dict(zip(bpcx_codebook['value'], bpcx_codebook['code']))

    # List to hold the reshaped tables
    long_format_tables = []

    # Iterate through each BPCX question
    for question in data.filter(regex='^'+q+"_").columns:

        ##ADDING WEIGHTS
        # Group by 'demo' and 'question', then apply the weighted counts calculation
        summary_table = data.groupby([demo,question]).apply(weighted_counts).unstack()

        # Rename index using the demo_labels dictionary
        summary_table = summary_table.rename(index=demo_labels)

        # Rename columns using the value_to_meaning dictionary
        summary_table = summary_table.rename(columns=value_to_meaning)

        # Reset index to make demo a column
        summary_table = summary_table.reset_index()
    #     display(summary_table)

        # Melt the DataFrame to long format
        melted = summary_table.melt(id_vars=demo, var_name='Response', value_name='Count')
        melted['Question'] = question
    #     display(melted)

        # Replace the question shorthand
        melted['Q_Text'] = q_codebook.get(question, question)  # Fallback to question ID if not found

        # Append to the list
        long_format_tables.append(melted)

    # Concatenate all the long format tables into one
    combined_long_table = pd.concat(long_format_tables)

    #remove "Other" open-text responses
    combined_long_table = combined_long_table[~combined_long_table["Question"].str.contains("TEXT")]

    #remove superfluous question text
    try:
        combined_long_table["Q_Text"] = combined_long_table["Q_Text"].str.split(' --- ', expand=True)[1].fillna(combined_long_table["Q_Text"])
    except:
        pass
    
    combined_long_table.drop(["Question"],axis=1,inplace=True)
    combined_long_table["Response"] = combined_long_table["Response"].str.replace('NO TEXT',"Selected")

    ##GET %s

    # Group by demo and 'Q_Text', then calculate the sum for each group
    total_counts = combined_long_table.groupby([demo, 'Q_Text'])['Count'].sum().reset_index(name='Total Count')

    # Filter only 'Selected' responses and calculate the sum
    selected_counts = combined_long_table[combined_long_table['Response'] == 'Selected'].groupby([demo, 'Q_Text'])['Count'].sum().reset_index(name='Selected Count')

    # Merge the total and selected counts on demo and 'Q_Text'
    merged_counts = pd.merge(total_counts, selected_counts, on=[demo, 'Q_Text'])

    # Calculate the percentage of 'Selected' for each demo and 'Q_Text'
    merged_counts['Selected Percentage'] = (merged_counts['Selected Count'] / merged_counts['Total Count'])
    counts = merged_counts[["Q_Text",demo,"Selected Count"]]
    counts = counts.pivot(index='Q_Text', columns=demo, values='Selected Count').reset_index()
    display(counts)
#     merged_counts.drop(["Total Count","Selected Count"],axis=1,inplace=True)
    merged_counts = merged_counts.pivot(index='Q_Text', columns=demo, values='Selected Percentage').reset_index()
    
    # Get total responses by Q_Text
    total_counts = combined_long_table.groupby(['Q_Text'])['Count'].sum().reset_index(name='Total Count')

    overall_percent = (combined_long_table.groupby(["Response","Q_Text"])['Count'].sum()/combined_long_table.groupby(['Q_Text'])['Count'].sum()[0]).reset_index()
    overall_percent.rename({"Count":"Overall"},axis=1,inplace=True)
    
    #merge
    overall_percent = overall_percent[overall_percent["Response"]=="Selected"]
    overall_percent.drop(["Response"],inplace=True,axis=1)
    
    merged_counts = pd.merge(merged_counts, overall_percent, on=['Q_Text'])
    display(merged_counts)

    merged_counts.to_csv("C://Users//rorey//OneDrive - Bipartisan Policy Center//Elections Project//Research//BPC SU II Joint Survey Dec 23//programming//files//"+q+"_"+demo+".csv",index=False)
#     counts.to_csv("C://Users//rorey//OneDrive - Bipartisan Policy Center//Elections Project//Research//BPC SU II Joint Survey Dec 23//programming//files//"+q+"_"+demo+"_COUNTS.csv",index=False)
    return(combined_long_table,merged_counts,overall_percent)

In [36]:
for q in ["BPC8"]:
    combined_long_table,merged_counts,overall_percent = get_demo_percents("age",q)

C:\Users\rorey\AppData\Local\Temp\ipykernel_15840\1993285230.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  overall_percent = (combined_long_table.groupby(["Response","Q_Text"])['Count'].sum()/combined_long_table.groupby(['Q_Text'])['Count'].sum()[0]).reset_index()


,Q_Text,Age: 18-34,Age: 35-44,Age: 45-64,Age: 65+,Overall
0,Apple Podcasts,0.051948,0.015542,0.042758,0.018849,0.035507
1,Discord,0.019614,0.005724,0.009994,0.004166,0.010742
2,Don't know/ no opinion,0.024306,0.065052,0.038664,0.036575,0.038540
3,Facebook,0.339122,0.438163,0.296029,0.179089,0.306510
4,Google,0.491380,0.476638,0.428317,0.341914,0.435441
5,I would not use online platforms to find elect...,0.058280,0.121335,0.270869,0.478171,0.230529
6,Instagram,0.182874,0.097300,0.100206,0.035778,0.109165
7,LinkedIn,0.034488,0.031508,0.033221,0.018613,0.030063
8,Nextdoor,0.012366,0.012750,0.006642,0.027224,0.013869
9,"Other, please specify",0.012618,0.026294,0.037031,0.034225,0.027596


In [64]:
# get_demo_percents("xpid3","BPC1")
# get_demo_percents("age","BPC1")

## Get over all %s (questions with 1 and 2 -- selected/not selected)

In [38]:
def get_weighted_response_totals(data, column, condition):
    """Calculate the sum of weights for responses meeting a specific condition."""
    # Use .loc to ensure the operation is done on the DataFrame, multiplying condition by weights
    return (data[column] == condition) * data['wts']

def get_total_percents(question):
    # Filter columns based on the question prefix
    bpc_columns = data.filter(regex='^'+question+'_').columns
    
    # Initialize dictionaries to store weighted totals and denominators
    weighted_totals = {}
    weighted_denoms = {}
    
    # Calculate weighted totals and denominators for each column
    for col in bpc_columns:
        weighted_totals[col] = get_weighted_response_totals(data, col, 1).sum()
        weighted_denoms[col] = get_weighted_response_totals(data, col, 1).sum() + get_weighted_response_totals(data, col, 2).sum()

    # Convert dictionaries to DataFrame
    BPC = pd.DataFrame(list(weighted_totals.items()), columns=['index', 'total'])
    
    # Calculate denominators for percentages and add them as a new column
    BPC['denominator'] = BPC['index'].map(weighted_denoms)
    
    # Replace question codes with text descriptions
    BPC["index"] = BPC["index"].map(q_codebook)
    try:
        BPC["index"] = BPC["index"].str.split(' --- ', expand=True)[1].fillna(BPC["index"])
    except:
        pass
    
    # Calculate percentages
    BPC["Percent"] = BPC["total"] / BPC['denominator']
    BPC.drop(["total","denominator"],axis=1,inplace=True)
    
    BPC.to_csv("C://Users//rorey//OneDrive - Bipartisan Policy Center//Elections Project//Research//BPC SU II Joint Survey Dec 23//programming//files//"+question+"_overall.csv",index=False)
    
    return BPC

In [39]:
for q in ["BPC1","BPC2","BPC9"]:
    get_total_percents(q)

In [44]:
for q in ["BPC11","BPC10"]:
    get_total_percents(q)

In [43]:
getdemo_scaled("xpid3","BPC4",False)

C:\Users\rorey\AppData\Local\Temp\ipykernel_15840\1706240069.py:99: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  overall_percent = (combined_long_table.groupby(["Response","Q_Text"])['Count'].sum()/combined_long_table.groupby(['Q_Text'])['Count'].sum()[0]).reset_index()


(                 xpid3                                           Response  \
 0   PID: Dem (no lean)        Very positive – I feel informed and engaged   
 1   PID: Ind (no lean)        Very positive – I feel informed and engaged   
 2   PID: Rep (no lean)        Very positive – I feel informed and engaged   
 3   PID: Dem (no lean)  Somewhat positive – I appreciate staying updat...   
 4   PID: Ind (no lean)  Somewhat positive – I appreciate staying updat...   
 5   PID: Rep (no lean)  Somewhat positive – I appreciate staying updat...   
 6   PID: Dem (no lean)  Neutral – I don't have strong feelings either way   
 7   PID: Ind (no lean)  Neutral – I don't have strong feelings either way   
 8   PID: Rep (no lean)  Neutral – I don't have strong feelings either way   
 9   PID: Dem (no lean)  Somewhat negative– It can be tiresome, but I s...   
 10  PID: Ind (no lean)  Somewhat negative– It can be tiresome, but I s...   
 11  PID: Rep (no lean)  Somewhat negative– It can be tiresome, 

## MISC  - get counts (not percent) for BPC12

In [48]:
combined_long_table,wide_format_table = getdemo_scaled("age","BPC12",False)

demo='age'

# Pivot the table to wide format
wide_format_table = combined_long_table.pivot_table(index=['Q_Text','Response'], 
                                                    columns=[demo], 
                                                    values='Count',
                                                    fill_value=0)

wide_format_table.reset_index(inplace=True)

for q_level in wide_format_table["Q_Text"].unique():
    # Mask for the rows that correspond to the question
    question_mask = wide_format_table["Q_Text"] == q_level

    # Select the relevant data for the current question
    temp = wide_format_table.loc[question_mask, wide_format_table.columns[2:]]

    # Calculate the totals for each column (for the current question)
    totals = temp.sum()

    # Use .loc to update the original dataframe
#     wide_format_table.loc[question_mask, temp.columns] = temp / totals.values

wide_format_table.to_csv("C://Users//rorey//OneDrive - Bipartisan Policy Center//Elections Project//Research//BPC SU II Joint Survey Dec 23//programming//files//BPC12counts.csv",index=False)

C:\Users\rorey\AppData\Local\Temp\ipykernel_15840\1706240069.py:99: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  overall_percent = (combined_long_table.groupby(["Response","Q_Text"])['Count'].sum()/combined_long_table.groupby(['Q_Text'])['Count'].sum()[0]).reset_index()


## BPC3 and BPC21

In [21]:
# GET importance levels for BPC21

combined_long_table,wide_format_table = getdemo_scaled("xpid3","BPC21",False)

# important = wide_format_table[(wide_format_table["Response"]=="Very important")|(wide_format_table["Response"]=="Somewhat important")]
important = wide_format_table[(wide_format_table["Response"]=="Very concerned")|(wide_format_table["Response"]=="Somewhat concerned")]

important = pd.DataFrame(important.groupby("Q_Text").value_counts()).reset_index()

important = important[['Q_Text', 'Response', 'Overall']]
important = important.pivot_table(index=['Q_Text'], columns=['Response'], 
                                                    values='Overall',
                                                    fill_value=0)

important.to_csv("C://Users//rorey//OneDrive - Bipartisan Policy Center//Elections Project//Research//BPC SU II Joint Survey Dec 23//programming//files//BPC21_important.csv",index=True)

# GET importance levels for BPC3

combined_long_table,wide_format_table = getdemo_scaled("xpid3","BPC3",False)

important = wide_format_table[(wide_format_table["Response"]=="Very important")|(wide_format_table["Response"]=="Somewhat important")]
# important = wide_format_table[(wide_format_table["Response"]=="Very concerned")|(wide_format_table["Response"]=="Somewhat concerned")]

important = pd.DataFrame(important.groupby("Q_Text").value_counts()).reset_index()

important = important[['Q_Text', 'Response', 'Overall']]
important = important.pivot_table(index=['Q_Text'], columns=['Response'], 
                                                    values='Overall',
                                                    fill_value=0)

important.to_csv("C://Users//rorey//OneDrive - Bipartisan Policy Center//Elections Project//Research//BPC SU II Joint Survey Dec 23//programming//files//BPC3_important.csv",index=True)

C:\Users\rorey\AppData\Local\Temp\ipykernel_15840\1706240069.py:99: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  overall_percent = (combined_long_table.groupby(["Response","Q_Text"])['Count'].sum()/combined_long_table.groupby(['Q_Text'])['Count'].sum()[0]).reset_index()
C:\Users\rorey\AppData\Local\Temp\ipykernel_15840\1706240069.py:99: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  overall_percent = (combined_long_table.groupby(["Response","Q_Text"])['Count'].sum()/combined_long_table.groupby(['Q_Text'])['Count'].sum()[0]).reset_index()


# GET TWO DEMOS: AGE AND PID FOR QUESTION 1

In [33]:
def weighted_counts(x):
    # x is a DataFrame of grouped values including the 'wts' column for weights
    total_weight = x['wts'].sum()
    return total_weight

q = "BPC1"
demo1 = 'age'
demo2 = 'xpid3'


# Assume demo1 and demo2 are your two demographic categories

# Extracting political party labels for both demographic categories
demo1_labels = dict(zip(codebook[codebook['question'] == demo1]['value'], codebook[codebook['question'] == demo1]['code']))
demo2_labels = dict(zip(codebook[codebook['question'] == demo2]['value'], codebook[codebook['question'] == demo2]['code']))

# Combining both sets of labels into a single dictionary for easier access later
demo_labels = {**demo1_labels, **demo2_labels}

# Filtering out the relevant rows from the codebook for BPCX responses remains the same
bpcx_codebook = codebook[codebook['question'].str.contains(q)]

# Creating a dictionary to map response values to their meanings also remains the same
value_to_meaning = dict(zip(bpcx_codebook['value'], bpcx_codebook['code']))

# List to hold the reshaped tables still needed
long_format_tables = []

# Iterate through each BPCX question as before
for question in data.filter(regex='^'+q+'_').columns:
    # Group by both 'demo1' and 'demo2' along with 'question', then apply the weighted counts calculation
    summary_table = data.groupby([demo1, demo2, question]).apply(weighted_counts).unstack()

    # Correctly setting levels for MultiIndex without using 'inplace'
    summary_table.index = summary_table.index.set_levels([summary_table.index.levels[0].map(demo1_labels.get),
                                                          summary_table.index.levels[1].map(demo2_labels.get)])
    
    # Rename columns using the value_to_meaning dictionary remains the same
    summary_table = summary_table.rename(columns=value_to_meaning)

    # Reset index to make both demos columns. This is adjusted to accommodate both demographic categories.
    summary_table = summary_table.reset_index()

    # Melt the DataFrame to long format, adjusting id_vars to include both demographic categories
    melted = summary_table.melt(id_vars=[demo1, demo2], var_name='Response', value_name='Count')
    melted['Question'] = question

    # Replace the question shorthand remains the same
    melted['Q_Text'] = q_codebook.get(question, question)  # Fallback to question ID if not found

    # Append to the list remains the same
    long_format_tables.append(melted)

# Concatenate all the long format tables into one remains the same
combined_long_table = pd.concat(long_format_tables)


In [48]:
# Remove "Other" open-text responses
try:
    combined_long_table = combined_long_table[~combined_long_table["Question"].str.contains("TEXT")]
except:
    pass

# Remove superfluous question text
try:
    combined_long_table["Q_Text"] = combined_long_table["Q_Text"].str.split(' --- ', expand=True)[1].fillna(combined_long_table["Q_Text"])
except:
    pass

try:
    combined_long_table.drop(["Question"], axis=1, inplace=True)
except:
    pass

combined_long_table["Response"] = combined_long_table["Response"].str.replace('NO TEXT', "Selected")

##GET %s

# Group by both demos and 'Q_Text', then calculate the sum for each group
total_counts = combined_long_table.groupby([demo1, demo2, 'Q_Text'])['Count'].sum().reset_index(name='Total Count')

# Filter only 'Selected' responses and calculate the sum
selected_counts = combined_long_table[combined_long_table['Response'] == 'Selected'].groupby([demo1, demo2, 'Q_Text'])['Count'].sum().reset_index(name='Selected Count')

# Merge the total and selected counts on both demos and 'Q_Text'
merged_counts = pd.merge(total_counts, selected_counts, on=[demo1, demo2, 'Q_Text'])

# Calculate the percentage of 'Selected' for each combination of demo1 and demo2 and 'Q_Text'
merged_counts['Selected Percentage'] = (merged_counts['Selected Count'] / merged_counts['Total Count'] * 100)  # Convert to percentage

merged_counts.drop(["Total Count", "Selected Count"], axis=1, inplace=True)

# To handle the two-dimensional structure, you might need to pivot or reshape data differently.
# If you want to pivot to have a multi-level column structure for demos, you can do something like this:
merged_counts_pivot = merged_counts.pivot_table(index='Q_Text', columns=[demo1, demo2], values='Selected Percentage').reset_index()

# Get total responses by Q_Text
total_counts_by_qtext = combined_long_table.groupby(['Q_Text'])['Count'].sum().reset_index(name='Total Count')

# Calculate overall percentages
overall_percent = (combined_long_table[combined_long_table['Response'] == 'Selected'].groupby(['Q_Text'])['Count'].sum() / total_counts_by_qtext['Total Count'][0]).reset_index(name='Selected Percentage')

# Flatten the MultiIndex columns
# Convert MultiIndex columns to a simple index by joining level names with an underscore (or any other separator you prefer)
merged_counts_pivot.columns = ['_'.join(col).strip() for col in merged_counts_pivot.columns.values]
merged_counts_pivot.rename({"Q_Text_":"Q_Text"},axis=1,inplace=True)

# Now, both DataFrames should have a single-level column structure
# Merge the overall percentages with the demo-based percentages
final_merged_counts = pd.merge(merged_counts_pivot, overall_percent, on=['Q_Text'], how='left')

# Display the final merged table
display(final_merged_counts)


final_merged_counts.to_csv("C://Users//rorey//OneDrive - Bipartisan Policy Center//Elections Project//Research//BPC SU II Joint Survey Dec 23//programming//files//BPC1_age&xpid.csv",index=True)

,Q_Text,Age: 18-34_PID: Dem (no lean),Age: 18-34_PID: Ind (no lean),Age: 18-34_PID: Rep (no lean),Age: 35-44_PID: Dem (no lean),Age: 35-44_PID: Ind (no lean),Age: 35-44_PID: Rep (no lean),Age: 45-64_PID: Dem (no lean),Age: 45-64_PID: Ind (no lean),Age: 45-64_PID: Rep (no lean),Age: 65+_PID: Dem (no lean),Age: 65+_PID: Ind (no lean),Age: 65+_PID: Rep (no lean),Selected Percentage
0,"AI-enabled chatbot (e.g., ChatGPT or Google's ...",3.629320,2.611728,3.835207,4.293053,0.342204,1.534772,2.259745,1.136422,2.070318,0.000000,0.000000,0.000000,0.019187
1,Don't know/ no opinion,2.845495,2.082941,2.527643,7.121196,6.002947,7.519291,0.487653,0.912469,3.712843,0.439850,1.948141,0.000000,0.025711
2,Friends and family,31.178228,40.913461,34.465458,32.713142,29.871759,31.132708,20.309135,15.891508,30.022357,18.524500,28.867390,29.807322,0.280262
3,I am not interested in election news or inform...,5.895119,5.641259,2.387959,2.369617,3.632902,3.842629,3.195800,10.920178,5.288568,3.502767,7.769855,2.395094,0.048546
4,Local or regional television,14.853330,14.137615,15.639134,23.659585,23.630978,31.258441,45.446195,36.692653,39.006106,50.896914,52.340029,56.258884,0.335605
5,National television,23.476302,28.576393,34.554428,34.947712,38.343623,27.100594,52.405535,43.693782,44.929986,67.641221,53.906061,56.999468,0.425382
6,"News aggregator apps (e.g., NewsBreak, SmartNe...",7.735058,7.459403,10.846169,16.021122,5.781820,6.856164,7.749932,16.794425,9.542779,6.816050,5.958837,7.593591,0.091268
7,News websites (including TV news and print pub...,24.338116,15.166372,26.387112,36.708548,23.946698,30.035056,34.971886,32.176503,30.535686,41.771459,40.811996,29.988371,0.303476
8,"Newsletters, blogs, or online forums",11.723977,8.341319,8.621457,6.178300,8.411244,2.700193,8.573919,3.659529,4.586671,3.483841,2.038270,0.989271,0.061860
9,"Other, please specify",0.000000,1.730459,0.000000,0.347077,0.000000,0.000000,0.339296,5.222016,3.046188,4.032565,0.475360,1.019621,0.015109
